# EDA

https://cogcomp.seas.upenn.edu/Data/QA/QC/definition.html                                        
https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label                                               
https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label

In [1]:
#petras_path for plots 
path_plot = '/home/petra42/AIDA_Abschluss/aida_question_classification/plots/'
#path_plot = '/home/an/aida_question_classification/plots/'

## Libaries

In [2]:
import pandas as pd
import numpy as np

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
path_class_def  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/definition.html'
path_train_data = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label'
path_test_data  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label'

In [ ]:
def process_question(row):
   return " ".join(row.split(" ")[1:])

train_df = pd.read_table(path_train_data, encoding = "ISO-8859-1", header=None)
train_df.columns = ["raw"]
train_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
train_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
train_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

train_df

In [ ]:
test_df = pd.read_table(path_test_data, encoding = "ISO-8859-1", header=None)
test_df.columns = ["raw"]
test_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
test_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
test_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

test_df

In [ ]:
train_df.head(5)

#describe
train_df.describe()
test_df.describe()


## question:
    - shape 
    - size
    - info: 
        count: row, unique categories, subcategories
    - witch catagories, subcategories, same in both dataframes
    - distribution
    - len questions (count token)
    

 

In [ ]:
# print test, train shape
print(f'---shapes---\ntrain:\t{train_df.shape}\ntest:\t{test_df.shape}')

In [ ]:
# number of row of columns
print(f'train_size:\t{train_df.size}\ntest_size:\t{test_df.size}') 

In [ ]:
# line occupancy    
print(f'---train---:\n {train_df.nunique()}\n')
print(f'---test---:\n {test_df.nunique()}')

In [ ]:
#which categories?
print(f"unique categories: {train_df['category'].unique()}")
#same categories in train and test?
train_df['category'].unique() == test_df['category'].unique()


In [ ]:
# load dictionary as dict
#which categories are not used?.
#which categories are used in train and test?

print(f"unique subcategories: {train_df['subcategory'].unique()}")





In [ ]:
# distribution categories (train)
dist_train_cat = train_df.groupby('category')['subcategory'].count()
print(dist_train_cat)
dist_train_cat.sort_values(ascending=False)

#visualisation 
dist_train_cat.plot.pie()
plt.title('Distribution categories (train)')
plt.savefig(path_plot + 'Distribution_categories_train.png')
plt.show()


In [ ]:
type(dist_train_cat)
dist_train_cat.values

In [ ]:
plt.pie(dist_train_cat,labels=dist_train_cat.index, autopct='%1.0f%%', pctdistance=1.15, labeldistance=1.3) 
plt.title('Distribution categories (train)')
plt.savefig(path_plot + 'Distribution_categories_train%.png')
plt.show()

In [ ]:
# Pie chart, where the slices will show the max_category:
labels=dist_train_cat.index
sizes = dist_train_cat
explode = (0, 0, 0.1, 0, 0, 0)  # only "explode" the 3nd slice (max_category)

fig1, ax1 = plt.subplots()
ax1.pie(dist_train_cat, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution categories (train)')
plt.savefig(path_plot + 'Distribution_categories_train_slice.png')
plt.show()

In [ ]:
# distribution categories (test)
dist_test_cat = test_df.groupby('category')['subcategory'].count()
dist_test_cat.sort_values(ascending=False)
print(dist_test_cat)

plt.pie(dist_test_cat,labels=dist_test_cat.index, autopct='%1.0f%%', pctdistance=1.15, labeldistance=1.3) 
plt.title('Distribution categories (test)')
plt.savefig(path_plot + 'Distribution_categories_test%.png')
plt.show()

In [ ]:
# Pie chart, where the slices will show the max_categories:
labels=dist_test_cat.index
explode = (0, 0.1, 0.1, 0, 0, 0)  # only "explode" the slice (max_categories)

fig1, ax1 = plt.subplots()
ax1.pie(dist_test_cat, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution categories (test)')
plt.savefig(path_plot + 'Distribution_categories_test_slice.png')
plt.show()

In [ ]:
# distribution subcategories (train)
dist_train_sub = train_df.subcategory.groupby(train_df['category']).value_counts()
print(dist_train_sub)

In [ ]:
#first 20 subcategories
print(f'first 20: {dist_train_sub.sort_values(ascending=False)[:20]}')

In [ ]:
fig= plt.figure(figsize=(12,6))
dist_train_sub.sort_values(ascending=False).plot.bar(label='train')
plt.xlabel('subcategories')
plt.ylabel('frequency')
plt.title('Distribution subcategories (train)')
plt.savefig(path_plot + 'Distribution_subcategories_train.png')
plt.show()

In [ ]:
# distribution subcategories (test)
dist_test_sub = test_df.subcategory.groupby(test_df['category']).value_counts()
print(dist_test_sub)


In [ ]:
print(f'first 20: {dist_train_sub.sort_values(ascending=False)[:20]}')

In [ ]:
plt.figure(figsize=(12,6))
dist_test_sub.sort_values(ascending=False).plot.bar(label='test')
plt.xlabel('subcategories')
plt.ylabel('frequency')
plt.title('Distribution subcategories (test)')
plt.savefig(path_plot + 'Distribution_subcategories_test.png')
plt.show()

In [ ]:
#plt.gca().set_xticklabels(xtickvals[::6], rotation=90, fontdict={'horizontalalignment': 'center', 'verticalalignment': 'center_baseline'})

In [ ]:
#test_df with only 38 subcategories compared to 47 in train_df
n = 14   #input for nlargest
train_df.subcategory.value_counts().nlargest(n).sum()

p = (train_df.subcategory.value_counts().nlargest(n).sum())/len(train_df)
print(f'The {n} largest subcategories account for a proportion of {p}')

In [ ]:
train_df.subcategory.value_counts().nlargest(n)

top_subcategories = list((train_df.subcategory.value_counts().nlargest(n)).index)
top_subcategories

train_df_top = train_df.loc[train_df['subcategory'].isin(top_subcategories)]
print(f'The {n} largest subcategories have {len(train_df_top)} questions.')

In [ ]:
train_df_top.sample(10)
train_df_top.subcategory.groupby(train_df['category']).value_counts()

#More than 80% of the data can be assigned to only 14 out of 47 subcategories in train_df

test_df.subcategory.value_counts().nlargest(39)

In [ ]:
def get_plot_length_text(df, title, diagram):
    '''
    creates a histogram 
    that shows the distribution of the number of tokens 
    per line for the data set to be examined
    
    Parameters
    ----------
    df:
        series; to examined text
    title: 
        string; title for Plot
    diagram: 
        string: name of the diagram
    
    Return
    ------
    Vizualisationplot Distribution
    '''
    plt.hist(df.apply(lambda text: len(text.split())))
    plt.xlabel('number of token')
    plt.ylabel('frequency')
    plt.title(title)
    plt.savefig(path_plot + diagram + '.png')
    plt.show()
    
    avg = round(df.apply(lambda text: len(text.split())).mean())
    maxi = max(list(df.apply(lambda text: len(text.split()))))
    mini = min(list(df.apply(lambda text: len(text.split()))))
    
    print(f'The texts have a mean length of {avg} tokens.')
    print(f'The longest text has {maxi} tokens, the shortest has {mini} tokens.')

In [ ]:
# distribution of length of questions, train data
df = train_df['question']
title = 'Distribution of length of questions (train)'
diagram ='length_questions_train'

get_plot_length_text(df, title, diagram)

In [ ]:
# distribution of length of questions, test data
df = test_df['question']
title = 'Distribution of length of questions (test)'
diagram ='length_questions_test'

get_plot_length_text(df, title, diagram)

In [ ]:
# distribution of length of questions, train and test data
ax1 = plt.hist(train_df['question'].apply(lambda text: len(text.split())),label='train')
ax2 = plt.hist(test_df['question'].apply(lambda text: len(text.split())), label='test')
plt.legend()
plt.xlabel('number of token')
plt.ylabel('frequencies per data set')
plt.title('Distributions of length of questions')
plt.savefig(path_plot + 'length_questions.png')
plt.show()

In [ ]:
#average length of question
al_train = round(train_df['question'].apply(lambda text: len(text.split())).mean())
al_test = round(test_df['question'].apply(lambda text: len(text.split())).mean())
print(f'The questions in the train data set have an average of {al_train} tokens,\
 and {al_test} tokens in the test data set.')

In [ ]:
#one token per cell --- from question
train_question_df = train_df['question'].str.split(" ", expand=True)
test_question_df = test_df['question'].str.split(" ", expand=True)

# print test, train shape
print(f'---shapes---\ntrain_df:\t{train_df.shape}\
\ntrain_question:\t{train_question_df.shape}\n\
test_question:\t{test_question_df.shape}')

train_question_df 

In [ ]:
#percentage distribution of tokens (train)
for column in train_question_df.columns:
    nan_sum = train_question_df[column].isnull().sum()
    print(column,': ',round(100.00-(nan_sum*100/len(train_question_df)),2))

In [ ]:
#percentage distribution of tokens (test)
for column in test_question_df.columns:
    nan_sum = test_question_df[column].isnull().sum()
    print(column,': ',round(100.00-(nan_sum*100/len(test_question_df)),2))

In [ ]:
#distribution of most frequent first words in questions - top 15
question_top_words = train_question_df[0].value_counts().nlargest(15)
question_top_words

In [ ]:
question_words = list(train_question_df[0].value_counts().nlargest(15).index[0:6])
question_words  #top 7 from above

In [ ]:
#combination of words table with (sub)categories

In [ ]:
train_df_words = train_df.join(train_question_df)

In [ ]:
train_df_words_top = train_df_words.loc[train_df_words[0].isin(question_words)]

In [ ]:
#More than 90% of the question texts start with one of 7 (top) question words
train_df_words_top.shape

In [ ]:
#question words give indications for category
train_df_words_top[0].groupby(train_df_words_top['category']).value_counts()

In [ ]:
#train_df_words_top   only used for eda

## Duplicates 

In [ ]:
#find duplicates in the raw text (category, subcategory and question)  ==> 100%match

def clean_duplicates(df, column):
    '''
    Find duplicates and drop by 100% match
    
    Parameters
    ----------
    df; name of the DataFrame
    column; to column to be examined 
    
    Output
    ------
    series; to check whether duplicates present
        df; clean df only duplicates
    '''
    duplicat = list(df[column].duplicated())
    print (set(duplicat))
    
    count_dups = df.pivot_table(index=['raw'], aggfunc='size')
    nb = len(df[column])-len(count_dups)
    print(f'The column has {nb} duplicates.')
    
clean_duplicates(train_df, 'raw')  


In [ ]:
#first value=> false, secound values =>true
dupl_first = list(train_df['raw'].duplicated())
print(list([index for index, value in enumerate(list(dupl_first)) if value == True]),'\n\n')

#first value=> true, secound values =>false
dupl_last = list(train_df['raw'].duplicated(keep='last'))
print(list([index for index, value in enumerate(list(dupl_last)) if value == True]))

In [ ]:
#sample (manueal) duplicates
train_df['raw'].loc[369]
train_df.loc[train_df['raw']=='NUM:speed What is the speed of the Mississippi River ?']

In [ ]:
train_df['raw'].loc[5219]
train_df.loc[train_df['raw']=='HUM:ind What Pope inaugurated Vatican International Radio ?']

In [ ]:
#remove duplicates
train_df_dropdup = train_df.drop_duplicates()
print(f'shape:\ntrain_df:\t\t{train_df.shape}\ntrain_df_droped:\t{train_df_dropdup.shape}')

In [ ]:
#check duplicates in test_df
clean_duplicates(test_df,'raw')

# Shrinkage of questions to shorter text


## nltk

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# default setting for lemmatizer and stopwords
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# remove stop words
nltk.download('stopwords')
my_stopwords = set(stopwords.words('english'))


def clean_text(text):
    """
    Regex cleaning of the text. Filters everything except alphanumerical and '.
    Return is turned into lower case

    Parameters
    ----------
    text : string
        text to be cleaned

    Returns
    -------
    string
        lower case regex cleaned text

    """
    text = text.replace("´", "'")
    
    text = text.replace("'s", " ")

    digi_punct = "[^a-zA-Z.1234567890#' ]"
    text = re.sub(digi_punct, " ", text)
    text = " ".join(text.split())
    text = text.lower()
    return text



def stopword_text(text):
    """
    Remove all words in the text that are in the stopword list
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text only stopwords
     
    """

    return " ".join([word for word in text.split() if word not in my_stopwords])



def lem_text(text):
    """
    Group the different inflected forms of a word so they can be analysed as 
    a single item
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text with lemmas
    """

    lem_sentence = text.split()
    for i, word in enumerate(text.split()):
        for pos in "n", "v", "a", "r":
            lem = lemmatizer.lemmatize(word, pos=pos)
            if lem != word:
                lem_sentence[i] = lem
                break
            else:
                lem_sentence[i] = word
    return " ".join(lem_sentence)


ModuleNotFoundError: No module named 'nltk'

In [ ]:
train_df.head(2)

In [ ]:
# create corpus
def corpus_func(df):
    '''
    create a textcorpus from pd.series 
    
    Parameters
    ----------
    text, string
    
    Return
    ------
    concatinated string with marker ##### as selector
    '''
    return "######".join(text for text in df)


In [ ]:
# corpus from df_train['question'] 
corpus = corpus_func(train_df['question'])
print(corpus[:500])

In [ ]:
#corpus from df_test['question']
corpus_test = corpus_func(test_df['question'])
print(corpus_test[:500])

In [ ]:
# create split-list corpus --> tranfer into list --> dataFrame

def corpus_list(corpus):
    '''split testcorpus in lines'''
    corpus_list = lambda x: x.split('######')

#try
#test = corpus_list(corpus)

## stopwords, clean_text (regex, lowercase)

In [ ]:
#check stopwords
stopword_text("why 's Peter's i I you what How Why go 's me how to school")  
#Attention: question words in LOWER case are in stopwords, in UPPER case not !!!

In [ ]:
# corpus df_train['text'] --> function stopwords
text_corpus = stopword_text(corpus) 
print(text_corpus[:500])


In [ ]:
#df_test['text']
text_corpus_test = stopword_text(corpus_test)
text_corpus_test[:100]

In [ ]:
#text column generated from question deleting stopwords
train_df['text'] = text_corpus.split('######')      
train_df.head(2)


In [ ]:
print(test_df.shape)
test_df.head(2)

In [ ]:
test_df['text'] = text_corpus_test.split('######') 

In [ ]:
# distribution of length of questions, after stopword-function
df = train_df['text']
title = 'Distribution of shortened text (train)'
diagram ='length_stop_train'

get_plot_length_text(df, title, diagram)

In [ ]:
train_text_df = train_df['text'].str.split(" ", expand=True)
train_text_df

test_text_df = test_df['text'].str.split(" ", expand=True)

In [ ]:
#percentage distribution of tokens (train)
for column in train_text_df.columns:
    nan_sum = train_text_df[column].isnull().sum()
    print(column,': ',round(100.00-(nan_sum*100/len(train_text_df)),2))

In [ ]:
set(train_text_df[16])

### clean_function

In [ ]:
# corpus df_train['text'] --> function clean_text
clean_corpus = clean_text(text_corpus)
print(clean_corpus[:500])

clean_corpus_test = clean_text(text_corpus_test)

In [ ]:
#text column generated from question deleting stopwords
train_df['text_clean'] = clean_corpus.split('######')      
train_df.head(2)

test_df['text_clean'] = clean_corpus_test.split('######') 

In [ ]:
# distribution of length of questions, after clean_function
df = train_df['text_clean']
title = 'Distribution of cleaned,shortened text (train)'
diagram ='length_clean_train'

get_plot_length_text(df, title, diagram)

In [ ]:
#train data
#one token per cell --- from text_clean
train_text_clean_df = train_df['text_clean'].str.split(" ", expand=True)
#test_text_clean_df = test_df['question'].str.split(" ", expand=True)

# print test, train shape
print(f'---shapes---\ntrain_df:\t\t{train_df.shape}\
\ntrain_text_clean:\t{train_text_clean_df.shape}')

train_text_clean_df 

In [ ]:
train_final_df = train_df.join(train_text_clean_df)
train_final_df.shape

In [ ]:
train_final_df.drop_duplicates(inplace=True)
train_final_df.head()

In [ ]:
train_final_df.shape

In [ ]:
#test data
#one token per cell --- from text_clean
test_text_clean_df = test_df['text_clean'].str.split(" ", expand=True)

# print test, train shape
print(f'---shapes---\ntest_df:\t\t{test_df.shape}\
\ntest_text_clean:\t{test_text_clean_df.shape}')

test_text_clean_df 

In [ ]:
test_final_df = test_df.join(test_text_clean_df)
test_final_df.shape

In [ ]:
test_final_df = test_df.join(test_text_clean_df)
print(test_final_df.shape)
test_final_df.head()


# end

In [ ]:
#test_question_df = test_df['question'].str.split(" ", expand=True)   tbd. to be cleaned with stopwords
#test_question_df

In [ ]:
# corpus df_test['text'] --> function stopwords
text_corpus_test = stopword_text(corpus_test) 
print(text_corpus_test[:500])

In [ ]:
# corpus df_train['text'] --> function clean_text
clean_corpus_test = clean_text(text_corpus_test)
print(clean_corpus_test[:500])


In [ ]:

test_df['text'] = text_corpus_test.split('######')      #text column generated from question deleting stopwords
test_df.head(2)

In [ ]:
# distribution of length of questions, test_df['text']
df = test_df['text']     # anzupassen
title = 'Distribution of shortened text (test)'
diagram ='length_short_test'

get_plot_length_text(df, title, diagram)

In [ ]:
#percentage distribution of tokens (test)
for column in test_question_df.columns:
    nan_sum = test_question_df[column].isnull().sum()
    print(column,': ',round(100.00-(nan_sum*100/len(test_question_df)),2))

In [ ]:
df_te= df_test.drop('text', clean, axis=1)

In [ ]:
#create corpus
def corpus_func(df, column):
    '''
    create a textcorpus from pd.series

    Parameters
    ----------
    text, string

    Return
    ------
    concatinated string with marker ##### as selector
    '''
    return "######".join(text for text in df)

def split_corpus_func(corpus):
    '''
    create a column from text corpus with marker '#####'
    as selector

    Parameters
    ----------
    text, string

    Return
    ------
    column in dafa frame
    '''
    return df

def preprocess_dataframe(df):
    '''
    create new columns in the data frame
    new_column: 'text' => cleaned stopwords (english)
                'text_clean' => regex, lowercase
                'text_lemma' => lemmetized
    param: df
    returns: df with new columns
    '''

    corpus = corpus_func(df['question'])
    text_corpus = stopword_text(corpus)
    df['text'] = split_corpus_func(corpus)
    clean_corpus = clean_text(text_corpus)
    df['text_clean'] = split_corpus_func(clean_corpus)
    lemma = lem_text(clean_corpus)
    df['text_lemma'] = split_corpus_func(clean_corpus)
    return df


In [4]:
preprocess_dataframe(df_te)
df_te

NameError: name 'preprocess_dataframe' is not defined